readme: Tasks include:
* use fasttext for word embedding
    * TODO: open pretrained word vectors
    * TODO: build word embedding matrix
    * TODO: modeling
* make predictions based on probability by modeling

In [1]:
import os
import numpy as np
import pandas as pd
os.chdir('/Users/liyuan/desktop/CSAir/codes')

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
from keras.layers import Flatten
from keras.layers import Embedding
import tensorflow as tf
from gensim.models import FastText

from prepare_data import PrepareData

Using TensorFlow backend.


In [12]:
class word2vec(object):
    def __init__(self):
        self.embeddings_index = {}
        self.MAX_SEQUENCE_LENGTH = 1000
        self.MAX_NUM_WORDS = 20000
        # embedding_dim need to be the same as the pretrained word vector dimensions
        self.EMBEDDING_DIM = 300    
        self.VALIDATION_SPLIT = 0.2
        self.all_labeled_data = pd.DataFrame()
        self.labels_index = {}
        self.word_index  = {}
        self.texts = np.array([])
        self.labels = np.array([])
        self.data = np.array([])
        
    def load_pretrained_vectors(self, file_path):
        f = open(file_path)
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            self.embeddings_index[word] = coefs
        f.close()
        print('Found %s word vectors.' % len(self.embeddings_index))
        return self.embeddings_index     
    
    def load_source_data(self,data_file_path):
        self.all_labeled_data = pd.read_csv(data_file_path)
        return self.all_labeled_data
    
    def prepare_data(self):
        # self.all_labeled_data = pd.read_csv(data_file_path)
        # self.texts = self.all_labeled_data.review_tokens.values
        self.texts = self.all_labeled_data.review_tokens.astype('str').values
        self.labels = self.all_labeled_data.label_encoded.values
        
        # get a dictionary that map each original label to its encoded label, e.g., {'中转': 0,...}
        for label in self.all_labeled_data.label.unique().tolist():
            self.labels_index[label] = self.all_labeled_data[self.all_labeled_data['label'] == label]['label_encoded'].unique()
        
        tokenizer = Tokenizer(nb_words=self.MAX_NUM_WORDS)
        tokenizer.fit_on_texts(self.texts)
        sequences = tokenizer.texts_to_sequences(self.texts)
        
        self.word_index = tokenizer.word_index
        print('Found %s unique tokens.' % len(self.word_index))
        
        self.data = pad_sequences(sequences, maxlen=self.MAX_SEQUENCE_LENGTH)
        print('Shape of data tensor:', self.data.shape)
        print('Shape of label tensor:', self.labels.shape)
        
        # Converts a class vector (integers) to binary class matrix
        self.labels = to_categorical(np.asarray(self.labels))
        print('Shape of data tensor:', self.data.shape)
        print('Shape of label tensor:', self.labels.shape)
        
        # split the data into a training set and a validation set
        indices = np.arange(self.data.shape[0])
        np.random.shuffle(indices)
        self.data = self.data[indices]
        self.labels = self.labels[indices]
        nb_validation_samples = int(self.VALIDATION_SPLIT * self.data.shape[0])
        
        self.X_train = self.data[:-nb_validation_samples]
        self.y_train = self.labels[:-nb_validation_samples]
        self.X_val = self.data[-nb_validation_samples:]
        self.y_val = self.labels[-nb_validation_samples:]
        return  self.X_train, self.y_train, self.X_val, self.y_val
    
    def get_embedding_matrix(self):
        self.embedding_matrix = np.zeros((len(self.word_index) + 1, self.EMBEDDING_DIM))
        for word, i in self.word_index.items():
            embedding_vector = self.embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                self.embedding_matrix[i] = embedding_vector
        return self.embedding_matrix
        

In [13]:
w2v = word2vec()
embeddings_index = w2v.load_pretrained_vectors('../Source_Data/sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5')

Found 635922 word vectors.


In [16]:
all_labeled_data = w2v.load_source_data('../res/labeled_data_with_without_tk.csv')
all_labeled_data.review_tokens.values

array(['生日 优惠券 领取 时间 下次 希望 标注 领取 时间', '参加 月 号 购 元 抵用券 未 收到 优惠券',
       '扑街 星巴克 合作 折 优惠 写上去 干 尼玛', ..., '月 乌鲁木齐 武汉 航班号 座位号 遗失 充电器 线 口袋',
       '分在 凤凰 机场 提供 接机 服务', '台北 发现自己 轮子 转盘 轮子 希望 贵司 托运 行李 时 温柔 点'],
      dtype=object)

In [17]:
# all_labeled_data = w2v.load_source_data('../res/all_labeled_data_v3.csv')
# all_labeled_data.review_tokens.values

X_train, y_train, X_val, y_val = w2v.prepare_data()
embedding_matrix = w2v.get_embedding_matrix()

Found 4747 unique tokens.
Shape of data tensor: (1551, 1000)
Shape of label tensor: (1551,)
Shape of data tensor: (1551, 1000)
Shape of label tensor: (1551, 10)


In [21]:
# TODO: get the validation set of review, true label and predicted label
nb_validation_samples = int(w2v.VALIDATION_SPLIT * w2v.data.shape[0])
print(nb_validation_samples)
w2v.data

310


array([[   0,    0,    0, ..., 2208,  453,  154],
       [   0,    0,    0, ...,  152,  257, 2107],
       [   0,    0,    0, ...,    0,    1, 2034],
       ...,
       [   0,    0,    0, ..., 2029,   68,    3],
       [   0,    0,    0, ..., 3890,  200,   30],
       [   0,    0,    0, ...,  155, 1900, 3162]], dtype=int32)